In [1]:
import collections
import matplotlib.pyplot as plt
import random
import pandas as pd
import numpy as np
import os
import sys
import anndata
# path = 'input/data/path/'
os.chdir(path)

In [2]:
# repeat this for all subjects, and the final results will be stored in out_path/ct/ci/NVQR_preds.csv. ci = int(100*(1-tau))

ct = 'AML328-D29'
tau = 0.10
d = 0
scaler='minmax'
# out_path = 'path/for/outfiles/'
out_path = path+'results/'
if not os.path.exists(out_path):
    os.makedirs(out_path)
command = 'python ./src/main.py  --dataset_name  aml_'+ct+'  --ds_type REAL  --tau '+str(tau)+'  --bs 256  --num_ep 10000  --calibration_ratio 0.3'+\
         '  --dropout 0.0  --hs "[256, 128, 64, 32]"  --pca 0  --softmax 0  --cali_factor '+str(d)+'  --seed 0  --scaler '+scaler+'  --suppress_plots 1 --out_dir ' + out_path + '\n'
os.system(command)
                   

out dir /home1/jiaweih/jiaweih_fsun/AML/10x_dataset/gdc_data/DeepDeconUQ/results/


INFO     | 2024-10-27 21:36:48,663 | __main__ | DEVICE: cpu


Y shape: torch.Size([30200, 2]), X shape: torch.Size([30200, 7362]), x log transform: True, y scaler: none
Y: tensor([[0.9887, 0.0113],
        [0.7993, 0.2007],
        [0.6867, 0.3133],
        [0.3193, 0.6807],
        [0.6653, 0.3347]])
x contain nan: False
scale: True, pca: 0, dan: False
train set size: x_train: torch.Size([18900, 7362]), y_train: torch.Size([18900, 2]), contain nan: 0
validation set size: x_val: torch.Size([3000, 7362]), y_val: torch.Size([3000, 2]), contain nan: 0
test set size: x_test: torch.Size([200, 7362]), y_te: torch.Size([200, 2]), contain nan: 0
dataset_name: aml_AML328-D29, tau: 0.1, conformalization tau: 0.1, seed: 0


INFO     | 2024-10-27 21:37:01,224 | __main__ | calibration set size: x_cal: torch.Size([8100, 7362]), y_cal: torch.Size([8100, 2])
  0%|          | 2/10000 [00:12<17:01:16,  6.13s/it]

In [3]:
# go to the stored file and store all malignant predictions in one file. specifically, open */ci/NVQR_preds.csv and extract malignant_lower and malignant_upper
# store them into one file name_deepdeconuq.txt
def get_com_coverage(ct, name):
    df1 = df.dropna()
    lower = df1[ct+ '_l'].values
    upper = df1[ct+ '_u'].values
    truth = df1['True'].values
    coverage = ((truth >= lower) & (truth <= upper)).sum()/len(df)
    mean_length = (upper - lower).mean()
    df_dict['Subject'].append(name)
    df_dict['Coverage'].append(coverage)
    df_dict['Average length'].append(mean_length)

subjects = ['AML328-D29', 'AML1012-D0', 'AML556-D0', 'AML328-D171', 
            'AML210A-D0', 'AML419A-D0', 'AML328-D0', 'AML707B-D0',
            'AML916-D0', 'AML328-D113', 'AML329-D0', 'AML420B-D0',
            'AML329-D20', 'AML921A-D0', 'AML475-D0', 
        ]
df_dict = {'Subject':[], 'Coverage':[], 'Average length':[]}
for name in subjects:
    df = pd.read_csv('./UQ/aml_perturbation/noise0/'+name+'_deepdeconuq.txt', index_col = 0)
    for ct in  ['ci90']:
        get_com_coverage(ct, name)
tmp = pd.DataFrame(df_dict).assign(Method='DeepDeconUQ')
comb = None
if comb is None:
    comb = tmp
else:
    comb = pd.concat([comb, tmp], axis = 0)
comb

,Subject,Coverage,Average length,Method
0,AML328-D29,0.905,0.045880,DeepDeconUQ
1,AML1012-D0,0.860,0.048687,DeepDeconUQ
2,AML556-D0,0.965,0.048260,DeepDeconUQ
3,AML328-D171,0.985,0.053917,DeepDeconUQ
4,AML210A-D0,0.895,0.037373,DeepDeconUQ
5,AML419A-D0,0.910,0.046264,DeepDeconUQ
6,AML328-D0,0.915,0.040309,DeepDeconUQ
7,AML707B-D0,0.925,0.052896,DeepDeconUQ
8,AML916-D0,0.895,0.047112,DeepDeconUQ
9,AML328-D113,0.990,0.055166,DeepDeconUQ
